In [10]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T
import numpy as np
import h5py

In [27]:
class EEGDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        super(EEGDataset).__init__()
        assert x.shape[0] == y.size
        self.x = x
        self.y = y
        
    def __getitem__(self,key):
        return (self.x[key], self.y[key])
    
    def __len__(self):
        return self.y.size

In [28]:
NUM_TRAIN = 700

transform = T.Compose([
                T.ToTensor()
            ])

f = h5py.File('child_mind_x_train.mat', 'r')
x_train = f['X_train']
x_train = np.reshape(x_train,(-1,1,24,256))
print('X_train shape: ' + str(x_train.shape))
f = h5py.File('child_mind_y_train.mat', 'r')
y_train = f['Y_train']
print('Y_train shape: ' + str(y_train.shape))
train_data = EEGDataset(x_train, y_train)

X_train shape: (729, 1, 24, 256)
Y_train shape: (729, 1)


In [37]:
train_data[700]

(array([[[13.298494  ,  9.598469  ,  7.911272  , ...,  1.5168967 ,
           3.5666392 , 16.765242  ],
         [ 6.6590223 , -6.0922527 ,  2.0290806 , ..., 20.829994  ,
          18.873888  , 25.210272  ],
         [22.702806  , 19.45708   , 12.724499  , ...,  0.22466445,
          -0.86936116, 17.228943  ],
         ...,
         [-0.6636641 , -1.048197  , 11.344592  , ..., -0.63276005,
           4.9658127 ,  0.68512547],
         [ 2.7453303 ,  6.2943773 ,  5.7231417 , ...,  7.7351646 ,
          14.930927  ,  9.094919  ],
         [ 8.087006  ,  7.5152664 ,  7.02515   , ...,  0.93873096,
           1.5594208 ,  9.94823   ]]], dtype=float32),
 array([0.]))

In [39]:
loader_train = DataLoader(train_data, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

In [4]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cpu


In [5]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [6]:
def train(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy(loader_val, model)
                print()

In [40]:
model = nn.Sequential(
                      nn.Conv2d(1,100,3),
                      nn.ReLU(),
                      nn.MaxPool2d(2, 2),
                      nn.Dropout(0.25),
                      nn.Conv2d(100,100,3),
                      nn.ReLU(),
                      nn.MaxPool2d(2, 2),
                      nn.Dropout(0.25),
                      nn.Conv2d(100,300,(2,3)),
                      nn.ReLU(),
                      nn.MaxPool2d(2, 2),
                      nn.Dropout(0.25),
                      nn.Conv2d(300,300,(1,7)),
                      nn.ReLU(),
                      nn.MaxPool2d((1,2), stride=1),
                      nn.Dropout(0.25),
                      nn.Conv2d(300,100,(1,3)),
                      nn.Conv2d(100,100,(1,3)),
                      nn.Flatten(),
                      nn.Linear(1900,6144),
                      nn.Linear(6144,2),
)

pred = model(next(iter(loader_train))[0])

In [41]:
print(pred.shape)

torch.Size([64, 2])


In [43]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)
train(model, optimizer)

RuntimeError: 1D target tensor expected, multi-target not supported

In [52]:
y_pred = model(x)
print("error rate: ")
print(np.sum(np.abs(np.argmax(y_pred, axis=1)-y))/y.size)

TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not numpy.ndarray